Code for generating sentences, starting with a language model. This differs in that it works on words -- with embeddings -- rather than on individual characters.


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class WordLanguageModelVectorizer(BaseEstimator, TransformerMixin):
    '''
    Base language model uses a CharacterEncoder to create character ordinals
    and then applies a transformation in order to create vectors.
    '''
    def __init__(self, context_length=64):
        '''
        Parameters
        ----------
        context_length : int
            This number of words will be used as a context to predict future words.
        '''
        self.context_length = context_length
        self.sequencer = CountVectorizer()
    
    def fit(self, strings):
        '''
        Fit the word vocabulary to target strings.
        
        Parameters
        ----------
        strings : iterable
            An iterable of source strings.
        '''
        # forgive passing a single string
        if type(strings) is str:
            strings = [strings]
        self.sequencer.fit(strings)
        self.sequencer.inverse_vocabulary_ = {sequence: word for word, sequence in self.sequencer.vocabulary_.items()}
        self.unique_words = len(self.sequencer.inverse_vocabulary_)
        return self
    
    def transform(self, strings):
        '''
        Transform strings into a (X, Y) pairing.
        
        Parameters
        ----------
        strings : iterable
            An iterable of source strings.
       
       Returns
        -------
        (np.ndarray, np.ndarray)
            A tuple (X, Y) three dimensional [sample_index, character_index] context X with a word sequence number
            to be embedded, and a two dimensional [sample_index, one_hot] target Y.
        '''
        # forgive passing a single string
        if type(strings) is str:
            strings = [strings]
        # start off by turning all the text into a series of integers
        word_sequence_numbers = []
        for string in strings:
            as_words = self.sequencer.build_analyzer()(string)
            word_sequence_numbers += list(map(self.sequencer.vocabulary_.get, as_words))
        # pad to the minimum context length
        if len(word_sequence_numbers) <= self.context_length:
            word_sequence_numbers = [0] * (1 + self.context_length - len(word_sequence_numbers)) + word_sequence_numbers
            
        # make this number of overlappinq sequences
        # ex with context 2: The quick brown fox likes chickens
        # The quick -> brown
        # quick brown -> fox
        number_of_contexts = len(word_sequence_numbers) - self.context_length
        # sequence numbers for context words
        x = np.zeros((number_of_contexts, self.context_length), dtype=np.int32)
        # one hot encodings for target words
        y = np.zeros((number_of_contexts, self.unique_words), dtype=np.bool)
        for i in range(number_of_contexts):
            context = np.array(word_sequence_numbers[i:i+self.context_length])
            x[i] = context
            target = word_sequence_numbers[i+self.context_length]
            y[i, target] = True
        return x, y
    
    def inverse_transform(self, X):
        '''
        Given a matrix of one hot encodings, reverse the transformation and return a matrix of characters.
        '''
        ordinals = X.argmax(-1)
        decoder = np.vectorize(self.sequencer.inverse_vocabulary_.get)
        # allow for single words or lists of words
        decoded = np.array([decoder(ordinals)])
        return ' '.join(decoded.flatten())

In [2]:
vectorizer = WordLanguageModelVectorizer()
with open('the_adventures_of_tom_sawyer.txt', encoding='utf8') as books:
    X, Y = vectorizer.fit_transform(books.readlines())

Let's take a look at the encoding as vectors.

In [3]:
X[0], Y[0], Y[0].argmax(-1)

(array([6770, 5133, 3068, 2111, 4580, 6770,  233, 4580, 6910, 5712, 1334,
         971, 4095, 7078, 5684, 1219, 6801, 2111, 3609, 2656, 6770, 7221,
        4580,  369,  374,  477, 4472, 1487,  341, 7598,  299, 4472, 5498,
        7487, 7729, 4126, 1470, 3616, 2884, 3616,  533, 4623, 5303, 7221,
        3616, 7124, 6770, 6746, 4580, 6770, 5133, 3068, 3890, 3465, 7598,
        6801, 2111, 4623, 4605,  477, 7675, 3068, 4443, 6885], dtype=int32),
 array([False, False, False, ..., False, False, False], dtype=bool),
 6770)

And -- inverse transformation, turning one-hots back into words!

In [4]:
vectorizer.inverse_transform(Y[0:100])

'the adventures of tom sawyer complete author mark twain samuel clemens release date august 20 2006 ebook 74 last updated june 2017 language english character set encoding utf start of this project gutenberg ebook tom sawyer produced by david widger the adventures of tom sawyer by mark twain samuel langhorne clemens contents chapter tom aunt polly decides upon her duty tom practices music the challenge private entrance chapter ii strong temptations strategic movements the innocents beguiled chapter iii tom as general triumph and reward dismal felicity commission and omission chapter iv mental acrobatics attending sunday school the superintendent showing off'

Now we can build up a recurrent neural network to learn a language model.

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, Dropout, Dense, Reshape, BatchNormalization, Embedding

class EmbeddedRecurrentLanguageModel(BaseEstimator):
    '''
    Create a language model with a neural network and normalized character encoding.
    '''

    def __init__(self, vectorizer, hidden_layers=256, gpu_optimized=False):
        '''
        Parameters
        ----------
        vectorizer : transformer
            Object to transform input strings into numerical encodings.
        hidden_layers : int
            Size of the model's hidden layer, controls complexity.
        gpu_optimized : bool
            If True, use special code in keras to boost performance.
        '''
        self.hidden_layers = hidden_layers
        self.gpu_optimized = gpu_optimized
        self.vectorizer = vectorizer

    def fit(self, strings, epochs=256, batch_size=256):
        '''
        Create and fit a model to the passed in strings.
        
        Parameters
        ----------
        strings : iterable
            An iterable source of string text.
        '''
        if self.gpu_optimized:
            RNN = CuDNNLSTM
        else:
            RNN = LSTM
        X, Y = self.vectorizer.fit_transform(strings)
        self.X = X
        self.Y = Y
        self.model = model = Sequential()
        # begin by embedding character positions
        model.add(Embedding(self.vectorizer.unique_words, self.hidden_layers, input_shape=(X.shape[1],)))
        # and then work on the embeddings recurrently
        model.add(RNN(self.hidden_layers, return_sequences=True))
        model.add(BatchNormalization())
        model.add(RNN(self.hidden_layers))
        model.add(BatchNormalization())
        model.add(Dense(self.hidden_layers, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(self.hidden_layers, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(Y.shape[1], activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        model.fit(X, Y, epochs=epochs, batch_size=batch_size)

Using TensorFlow backend.
/home/wballard/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Run this model with cities. I'm using a GPU - and very much recommend you do so! You can set the optimization to False if you need to use a CPU.

City names aren't sentences, so we need to use a relatively short context length.

In [6]:
vectorizer = WordLanguageModelVectorizer()
model = EmbeddedRecurrentLanguageModel(vectorizer, gpu_optimized=True)
with open('the_adventures_of_tom_sawyer.txt', encoding='utf8') as books:
    model.fit(books.readlines())

Epoch 1/256
72635/72635 [==============================] - 298s 4ms/step - loss: 7.3194
Epoch 2/256
72635/72635 [==============================] - 18s 242us/step - loss: 5.8302
Epoch 3/256
72635/72635 [==============================] - 18s 243us/step - loss: 5.3088
Epoch 4/256
72635/72635 [==============================] - 18s 242us/step - loss: 4.8433
Epoch 5/256
72635/72635 [==============================] - 18s 242us/step - loss: 4.3602
Epoch 6/256
72635/72635 [==============================] - 17s 241us/step - loss: 3.8395
Epoch 7/256
72635/72635 [==============================] - 18s 242us/step - loss: 3.3049
Epoch 8/256
72635/72635 [==============================] - 18s 242us/step - loss: 2.7772
Epoch 9/256
72635/72635 [==============================] - 18s 244us/step - loss: 2.2998
Epoch 10/256
72635/72635 [==============================] - 18s 243us/step - loss: 1.8813
Epoch 11/256
72635/72635 [==============================] - 18s 243us/step - loss: 1.5132
Epoch 12/256
72635/7

72635/72635 [==============================] - 17s 237us/step - loss: 0.0025
Epoch 93/256
72635/72635 [==============================] - 17s 238us/step - loss: 0.0019
Epoch 94/256
72635/72635 [==============================] - 17s 237us/step - loss: 0.0015
Epoch 95/256
72635/72635 [==============================] - 17s 238us/step - loss: 0.0013
Epoch 96/256
72635/72635 [==============================] - 17s 238us/step - loss: 0.0012
Epoch 97/256
72635/72635 [==============================] - 17s 239us/step - loss: 0.0015
Epoch 98/256
72635/72635 [==============================] - 17s 238us/step - loss: 0.0019
Epoch 99/256
72635/72635 [==============================] - 17s 238us/step - loss: 0.0804
Epoch 100/256
72635/72635 [==============================] - 17s 236us/step - loss: 0.8513
Epoch 101/256
72635/72635 [==============================] - 17s 236us/step - loss: 0.1681
Epoch 102/256
72635/72635 [==============================] - 17s 239us/step - loss: 0.0332
Epoch 103/256
72635/

72635/72635 [==============================] - 17s 236us/step - loss: 7.0467e-04
Epoch 182/256
72635/72635 [==============================] - 17s 237us/step - loss: 6.6953e-04
Epoch 183/256
72635/72635 [==============================] - 17s 236us/step - loss: 6.4749e-04
Epoch 184/256
72635/72635 [==============================] - 17s 236us/step - loss: 6.2605e-04
Epoch 185/256
72635/72635 [==============================] - 17s 236us/step - loss: 6.0193e-04
Epoch 186/256
72635/72635 [==============================] - 17s 235us/step - loss: 5.8364e-04
Epoch 187/256
72635/72635 [==============================] - 17s 235us/step - loss: 5.6708e-04
Epoch 188/256
72635/72635 [==============================] - 17s 235us/step - loss: 5.5415e-04
Epoch 189/256
72635/72635 [==============================] - 17s 235us/step - loss: 0.7499
Epoch 190/256
72635/72635 [==============================] - 17s 235us/step - loss: 0.3239
Epoch 191/256
72635/72635 [==============================] - 17s 236us/s

In [7]:
model.vectorizer = WordLanguageModelVectorizer()
with open('the_adventures_of_tom_sawyer.txt', encoding='utf8') as books:
    X, Y = model.vectorizer.fit_transform(books.readlines())

Now for generation. Any words you like -- which will be padded as needed.

In [8]:
class SentenceLanguageModelGenerator():
    '''
    Given a language model, generate new name strings given a seed of your own design.
    '''
    
    def __init__(self, language_model):
        '''
        Parameters
        ----------
        language_model
            A trained language model used to generate predictions.
        '''
        self.language_model = language_model
        
    def generate(self, seed, max_length=64):
        '''
        Parameters
        ----------
        seed : str
            A string to bootstrap generation.
        max_length: int
            A guard value to prevent looping forever.
        '''
       
        # build up the result buffer here, adding on to our passed seed
        result = seed
        for i in range(0, max_length):
            # working on the right most context
            X, _ = self.language_model.vectorizer.transform([seed])
            context = np.array([X[-1]])
            # only need the very first sample, then keep iterating
            try:
                prediction = self.language_model.model.predict(context)[0]
                next_word = self.language_model.vectorizer.inverse_transform(prediction)
            except IndexError:
                # when we hit a null character, it is time to exit
                break
            # keep expanding the seed with each word
            seed += ' ' + next_word
        return seed

In a sense, this is a kind of a machine learning made up autocomplete, we'll start with a few words and see what it tacks on!

In [9]:
SentenceLanguageModelGenerator(model).generate('I would like to understand all of the ways')

'I would like to understand all of the ways country summer mighty right grave in in our front secret of for injun ever joe have harper been always away built he or another stop kite tom strings had stick presence to with work one of sort them of men men were were gone in and the late afternoon tavern after and breakfast they although presently the the book welshman almost and went they'